In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from IPython import get_ipython
#%%
get_ipython().run_line_magic('matplotlib', 'qt5')
cm=1/2.54
#%%


# Ruta de la carpeta
folder_path = r"C:\Users\otero\Downloads\histeresis mezcla"
files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])

# Lista para almacenar la primera fila de la columna 5 de cada archivo
first_row_col5 = []

# Configuración del gráfico
plt.figure(figsize=(16*cm, 12*cm))


# Procesar cada archivo
for file_name in files:
    # Ruta completa del archivo
    file_path = os.path.join(folder_path, file_name)

    # Cargar los datos del archivo
    data = pd.read_csv(file_path, delimiter=",", skiprows=1)

    # Extraer la primera fila de la columna 5 (índice 4 en Python)
    first_row_col5.append(data.iloc[0, 4])

    # Extraer las columnas según los nombres dados
    tiempo = data.iloc[:, 0]
    amplitud_ent = data.iloc[:, 1]
    amplitud_int = data.iloc[:, 2]

    # Graficar amplitud_int en función de amplitud_ent

    plt.plot(amplitud_ent, amplitud_int)


# Añadir leyenda y mostrar el gráfico
#plt.legend()
plt.xlabel("$\mathrm{V}_{\mathrm{A}}$[V]",fontsize=12)
plt.ylabel("$\mathrm{V}_{\mathrm{B}}$[V]",fontsize=12)
#plt.legend()
plt.grid(True)

plt.show()

# Imprimir la primera fila de la columna 5 para cada archivo
print("Primera fila de la columna 5 de cada archivo:")
for i, value in enumerate(first_row_col5):
    print(f"Archivo {files[i]}: Valor = {value}")

#%%
res=np.divide(np.array(np.sort(first_row_col5)),10000)
#%%
"""
Tabla de calibracion de un PT100
Rango Temperatura: -200C a 850C
Rango Resistencia: 18.52Ohm a 390.48Ohm
"""

# Funcion que carga tabla e interpola
def PT100_res2temp_interp(R): #en Ohm
    data = np.loadtxt('Pt100_resistencia_temperatura.csv',delimiter=',')
    temperature_vals = data[:,0] # en Celsius
    resistance_vals = data[:,1] #en Ohm
    return np.interp(R, resistance_vals, temperature_vals)

R = np.linspace(18.53,390.47,100)
plt.plot(R, PT100_res2temp_interp(R))
plt.ylabel('Temperatura [C]')
plt.xlabel('Resistencia Pt100 [Ohm]')
plt.grid(True)

temperatures = [PT100_res2temp_interp(R) for R in res]

# Imprimir los resultados
print("Temperaturas correspondientes a cada resistencia:")
for R, T in zip(res, temperatures):
    print(f"Resistencia: {R} Ω -> Temperatura: {T:.2f} °C")
#%%

# Parámetros
folder_path = r"C:\Users\otero\Downloads\histeresis placa 5"
threshold = 0.05  # Umbral para considerar amplitud_ent ~ 0

# Obtener y ordenar los archivos por el número inicial en sus nombres
files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith('.csv')],
    key=lambda x: int(x.split('_')[0])
)

# Lista para almacenar los valores máximos de amplitud_int cuando amplitud_ent ~ 0
max_amplitud_int_at_zero_ent = []
 # Configuración del gráfico
plt.figure(figsize=(10, 6))
plt.title("Amplitud_int en función de Amplitud_ent")
plt.xlabel("Amplitud_ent")
plt.ylabel("Amplitud_int")

# Procesar cada archivo
for file_name in files:
    # Ruta completa del archivo
    file_path = os.path.join(folder_path, file_name)


    # Cargar los datos del archivo
    data = pd.read_csv(file_path, delimiter=",", skiprows=1)

    # Extraer las columnas según los nombres dados
    amplitud_ent = data.iloc[:, 1]
    amplitud_int = data.iloc[:, 2]
    plt.scatter(amplitud_ent, amplitud_int, label=f'{file_name}')
    # Encontrar los valores de amplitud_int donde amplitud_ent ~ 0
    mask = (np.abs(amplitud_ent) <= threshold) & (amplitud_int > 0)
    if mask.any():
        # Obtener el valor máximo de amplitud_int donde amplitud_ent ~ 0
        max_amplitud_int = amplitud_int[mask].max()
        max_amplitud_int_at_zero_ent.append((file_name, max_amplitud_int))
    else:
        max_amplitud_int_at_zero_ent.append((file_name, None))  # No encontrado


# Añadir leyenda y mostrar el gráfico
#plt.legend()
plt.show()
# Imprimir resultados
print("Valores máximos de amplitud_int para amplitud_ent cercano a 0:")
for file_name, max_amp_int in max_amplitud_int_at_zero_ent:
    if max_amp_int is not None:
        print(f"{file_name}: Max amplitud_int = {max_amp_int}")
    else:
        print(f"{file_name}: No se encontró valor de amplitud_ent cercano a 0")
#%%import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

# Ruta de la carpeta
folder_path = r"C:\Users\otero\Downloads\histeresis placa 5"

# Obtener y ordenar los archivos por el número en su nombre
files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith('.csv')],
    key=lambda x: int(re.search(r'\d+', x).group())
)

# Definir tolerancia para amplitud_ent cercano a cero
tolerance = 0.1

# Listas para almacenar los valores de cada archivo
amplitud_ent_cero_vals = []
amplitud_int_max_vals = []

# Configurar el gráfico
plt.figure(figsize=(10, 6))
plt.title("Amplitud_int en función de Amplitud_ent")
plt.xlabel("Amplitud_ent")
plt.ylabel("Amplitud_int")

# Procesar cada archivo
for file_name in files:
    # Cargar los datos del archivo
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path, delimiter=",", skiprows=1)

    # Extraer columnas
    amplitud_ent = data.iloc[:, 1]
    amplitud_int = data.iloc[:, 2]

    # Filtrar los puntos en los que amplitud_ent está cerca de cero
    mask = np.abs(amplitud_ent) < tolerance
    if np.any(mask):
        # Seleccionar el valor máximo de amplitud_int dentro de los valores cercanos a cero
        amplitud_int_max = amplitud_int[mask].max()
        amplitud_int_max_vals.append(amplitud_int_max)

        # Guardar el valor correspondiente de amplitud_ent cercano a cero
        amplitud_ent_cero = amplitud_ent[mask].iloc[amplitud_int[mask].idxmax()]
        amplitud_ent_cero_vals.append(amplitud_ent_cero)

    # Graficar los datos con error bars
    plt.errorbar(amplitud_ent, amplitud_int, yerr=0.01, fmt='-', label=file_name)  # Ajusta 'yerr' según sea necesario

# Mostrar gráfico y leyenda
plt.legend()
plt.show()

# Imprimir resultados
print("Valores máximos de Amplitud_int cerca de Amplitud_ent = 0 para cada archivo:")
for file_name, ent_cero, int_max in zip(files, amplitud_ent_cero_vals, amplitud_int_max_vals):
    print(f"{file_name}: Amplitud_ent cercano a 0 = {ent_cero}, Amplitud_int máximo = {int_max}")

#%%
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

# Ruta de la carpeta
folder_path = r"C:\Users\otero\Downloads\histeresis mezcla"

# Obtener y ordenar los archivos por el número en su nombre
files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith('.csv')],
    key=lambda x: int(re.search(r'\d+', x).group())
)

# Configuración del gráfico
plt.figure(figsize=(10, 6))
plt.title("Amplitud_int en función de Amplitud_ent")
plt.xlabel("Amplitud_ent")
plt.ylabel("Amplitud_int")

# Lista para almacenar los valores de "amplitud_int" máximos cercanos a amplitud_ent = 0
amplitud_int_max_ceros = []

# Procesar cada archivo
for file_name in files:
    # Ruta completa del archivo
    file_path = os.path.join(folder_path, file_name)

    # Cargar los datos del archivo
    data = pd.read_csv(file_path, delimiter=",", skiprows=1)

    # Extraer las columnas necesarias
    amplitud_ent = data.iloc[:, 1]
    amplitud_int = data.iloc[:, 2]

    # Definir el intervalo alrededor de amplitud_ent = 0
    intervalo = 0.1
    mask = (amplitud_ent >= -intervalo) & (amplitud_ent <= intervalo)

    # Comprobar si hay valores en el intervalo
    if mask.any():
        # Encontrar el valor de amplitud_int máximo dentro del intervalo
        max_amplitud_int = amplitud_int[mask].max()
        amplitud_ent_cero = amplitud_ent[mask][amplitud_int[mask].idxmax()]
        amplitud_int_max_ceros.append((file_name, amplitud_ent_cero, max_amplitud_int))

        # Graficar con error bars
        plt.errorbar(amplitud_ent, amplitud_int, yerr=0.01, label=f'{file_name}', fmt='o')

# Añadir leyenda y mostrar el gráfico
#plt.legend()
plt.show()

# Imprimir los valores de "amplitud_int" máximos cercanos a amplitud_ent = 0 para cada archivo
print("Valores máximos de amplitud_int cercanos a amplitud_ent = 0:")
for file_name, ent_cero, int_max in amplitud_int_max_ceros:
    print(f"Archivo {file_name}: Amplitud_ent = {ent_cero}, Amplitud_int max = {int_max}")



#%%import os
import os
import pandas as pd
import numpy as np

# Define la carpeta y carga los archivos de forma ordenada
folder_path = r"C:\Users\otero\Downloads\histeresis mezcla"
# Ordena archivos extrayendo solo el número en el nombre y convirtiéndolo en entero
files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')], key=lambda x: int(x.split('.')[0]))

# Inicializa listas para almacenar los valores
amplitud_ent_cero_vals_posta = []
amplitud_int_max_vals_posta = []
col_5 = []



# Procesa cada archivo
for file_name in files:
    # Ruta completa del archivo
    file_path = os.path.join(folder_path, file_name)

    # Carga los datos del archivo, omitiendo la primera fila con los títulos
    data = pd.read_csv(file_path, delimiter=",", skiprows=1)

    # Extrae las columnas necesarias
    amplitud_ent = data.iloc[:, 1]
    amplitud_int = data.iloc[:, 2]
    col_5.append(data.iloc[0, 4])
    # Máscara para valores cercanos a amplitud_ent = 0
    mask = (amplitud_ent >= -0.1) & (amplitud_ent <= 0.1)

    # Verifica si hay valores en el rango deseado
    if mask.any() and not amplitud_int[mask].empty:
        # Encuentra el valor máximo de amplitud_int dentro del intervalo
        max_amplitud_int_in_range = amplitud_int[mask].max()
        # Obtén el valor correspondiente de amplitud_ent usando el índice del valor máximo
        idx_max = amplitud_int[mask].idxmax()
        if idx_max in amplitud_ent[mask].index:  # Verificar que el índice esté dentro del rango
            amplitud_ent_cero = amplitud_ent[mask].loc[idx_max]
        else:
            amplitud_ent_cero = np.nan

        # Guarda los valores en las listas
        amplitud_ent_cero_vals_posta.append(amplitud_ent_cero)
        amplitud_int_max_vals_posta.append(max_amplitud_int_in_range)
    else:
        # Si no hay valores en el rango, agregar NaN
        amplitud_ent_cero_vals.append(np.nan)
        amplitud_int_max_vals.append(np.nan)

# Ahora amplitud_ent_cero_vals y amplitud_int_max_vals contienen los datos deseados
print("Valores de amplitud_ent_cero:", amplitud_ent_cero_vals_posta)
print("Valores de amplitud_int_max:", amplitud_int_max_vals_posta)
#%%
res_posta=np.divide(np.array(np.sort(col_5)),10000)
amplitudes_posta=np.sort(amplitud_int_max_vals_posta)[::-1]
# Funcion que carga tabla e interpola
def PT100_res2temp_interp(R): #en Ohm
    data = np.loadtxt('Pt100_resistencia_temperatura.csv',delimiter=',')
    temperature_vals = data[:,0] # en Celsius
    resistance_vals = data[:,1] #en Ohm
    return np.interp(R, resistance_vals, temperature_vals)

R = np.linspace(18.53,390.47,100)
plt.plot(R, PT100_res2temp_interp(R))
plt.ylabel('Temperatura [C]')
plt.xlabel('Resistencia Pt100 [Ohm]')
plt.grid(True)

temperatures_posta = [PT100_res2temp_interp(R) for R in res_posta]

# Imprimir los resultados
print("Temperaturas correspondientes a cada resistencia:")
for R, T in zip(res, temperatures):
    print(f"Resistencia: {R} Ω -> Temperatura: {T:.2f} °C")

#%%
temperatures_posta=np.array(temperatures_posta)
temp_usar=temperatures_posta[24:55]
amp_usar=amplitudes_posta[24:55]
error_amp_usar=np.divide(3*amp_usar,100)
error_temp_usar=np.abs(np.divide((1/2)*temp_usar,100))

temp_no_usar = np.concatenate((temperatures_posta[:24], temperatures_posta[55:]))
amp_no_usar = np.concatenate((amplitudes_posta[:24], amplitudes_posta[55:]))

error_amp_no_usar=np.divide(3*amp_no_usar,100)
error_temp_no_usar=np.abs(np.divide(0.5*temp_no_usar,100))


plt.figure(754)
plt.scatter(temp_usar,amp_usar,color="blue")
#%%
def exponencial(x,tc,beta,a):
  return a*(tc-x)**beta
#%%
params,cov=curve_fit(exponencial,temp_usar,amp_usar,p0=[-25,0.30,0.01],sigma=error_amp_usar)
tc=params[0]
beta=params[1]
a=params[2]
errores_params=np.sqrt(np.diag(cov))
error_tc=errores_params[0]
error_beta=errores_params[1]

print(tc),print(error_tc)
print(beta),print(error_beta)

#%%
plt.figure(267,figsize=(16*cm,12*cm))
plt.errorbar(temp_usar,amp_usar,xerr=error_temp_usar,yerr=error_amp_usar,color="red",fmt=".",label="Valores usados en el ajuste")
plt.errorbar(temp_no_usar,amp_no_usar,xerr=error_temp_no_usar,yerr=error_amp_no_usar,fmt=".",label="Valores no usados en el ajuste")
plt.plot(temperatures_posta[:55],exponencial(temperatures_posta[:55],tc,beta,a),color="orange",zorder=1000,label="Ajuste")
plt.ylabel("Señal proporcional a $\mathrm{M}_{\mathrm{R}}$ [V]",fontsize=12)
plt.xlabel("Temperatura [°C]",fontsize=12)
plt.legend()
plt.grid(True)

plt.show()
#%%
plt.figure(268,figsize=(16*cm,12*cm))
plt.errorbar(temp_usar,amp_usar,xerr=error_temp_usar,yerr=error_amp_usar,color="red",fmt=".",label="valores usados en el ajuste")
plt.plot(temp_usar,exponencial(temp_usar,tc,beta,a),color="orange",zorder=1000,label="ajuste")
plt.ylabel("Señal proporcional a $\mathrm{M}_{\mathrm{R}}$ [V]",fontsize=12)
plt.xlabel("Temperatura [°C]",fontsize=12)
plt.legend()
plt.grid(True)

plt.show()
#%%
y_pred = exponencial(temp_usar,tc,beta,a)  # El "y" predicho a partir del ajuste
residuos = amp_usar - y_pred
grados_libertad=len(amp_usar)-3-1
chi_cuadrado = np.sum(((amp_usar-y_pred)/error_amp_usar)**2)
chi_cuadrado_reducido = chi_cuadrado/grados_libertad
# interpretamos el resultado:
print('Grados de libertad: ' + str(grados_libertad))
print('chi^2: ' + str(chi_cuadrado))
print('chi^2 reducido: ' + str(chi_cuadrado_reducido))

# aplicamos algún criterio:

if error_amp_usar[0]==0:
    print('No se declararon errores en la variable y.')
elif chi_cuadrado < grados_libertad - 1.65*np.sqrt(2*grados_libertad): # esto es APROX el criterio del 5%
    print('Probablemente se esté sobreajustando, o quizás los errores están sobreestimados.')
elif chi_cuadrado > grados_libertad + 1.65*np.sqrt(2*grados_libertad): # esto es APROX el criterio del 5%
    print('Probablemente deba rechazarse la hipótesis de que el modelo ajuste a los datos, o quizás los errores están subestimados.')
else:
    print('Probablemente no pueda rechazarse la hipótesis de que el modelo ajuste a los datos.')
#%%
from matplotlib.gridspec import GridSpec

# Crear la figura y configurar GridSpec
fig = plt.figure(figsize=(16 * cm, 12 * cm))
gs = GridSpec(2, 1, height_ratios=[8, 4], figure=fig)  # Proporciones de altura 8:4

# Subplot 0: Datos y ajuste
ax0 = fig.add_subplot(gs[0])
ax0.errorbar(
    temp_usar, amp_usar,
    xerr=error_temp_usar, yerr=error_amp_usar,
    color="red", fmt=".", label="valores usados en el ajuste"
)
ax0.plot(
    temp_usar, exponencial(temp_usar, tc, beta, a),
    color="orange", zorder=1000, label="ajuste"
)
ax0.set_ylabel("Señal proporcional a $\mathrm{M}_{\mathrm{R}}$ [V]", fontsize=12)
ax0.legend()
ax0.grid(True)

# Quitar los ticks del eje X en el subplot 0
ax0.tick_params(labelbottom=False)

# Añadir etiqueta "a" al subplot 0
ax0.text(0.02, 0.75, "(a)", transform=ax0.transAxes, fontsize=14, fontweight="bold", va="top", ha="left")

# Subplot 1: Residuos
ax1 = fig.add_subplot(gs[1], sharex=ax0)
ax1.errorbar(
    temp_usar, residuos,
    xerr=error_temp_usar, yerr=error_amp_usar,
    fmt=".", color="red", label="residuos"
)
ax1.axhline(0, color="black", linestyle="--", linewidth=1, zorder=500)
ax1.set_xlabel("Temperatura [°C]", fontsize=12)
ax1.set_ylabel("Residuos[V]", fontsize=12)
ax1.legend()
ax1.grid(True)

# Añadir etiqueta "b" al subplot 1
ax1.text(0.02, 0.95, "(b)", transform=ax1.transAxes, fontsize=14, fontweight="bold", va="top", ha="left")

# Ajustar el espacio entre subplots
fig.tight_layout()

# Mostrar el gráfico
plt.show()

